In [73]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [16]:
data = pd.read_csv("C:\\Users\\Bala Vignesh.A\\Desktop\\SCMA 632\\NSSO68.csv", low_memory=False)

In [31]:
display (data)

,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot,non_veg
0,1,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.180000,1
1,2,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.950000,1
2,3,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.840000,1
3,4,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.300000,1
4,5,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101657,101658,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.000000,544.013667,28.441750,J$K,2,0.000000,25.833333,1
101658,101659,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,8.000000,417.616600,25.490282,J$K,2,0.000000,49.000000,1
101659,101660,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,7.142857,378.300429,25.800107,J$K,2,0.000000,32.285714,1
101660,101661,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,14.000000,510.023600,30.220170,J$K,2,0.000000,39.200000,1


# Filter the data for non-vegetarians

In [33]:
non_veg_values = non_veg_data['non_veg']
print(non_veg_values)

0         1
1         1
2         1
3         1
4         1
         ..
101657    1
101658    1
101659    1
101660    1
101661    1
Name: non_veg, Length: 101047, dtype: int64


In [34]:
print(non_veg_values.unique())

[1]


In [35]:
print(non_veg_values.value_counts())

non_veg
1    101047
Name: count, dtype: int64


In [40]:
print(data.columns)

Index(['slno', 'grp', 'Round_Centre', 'FSU_number', 'Round', 'Schedule_Number',
       'Sample', 'Sector', 'state', 'State_Region',
       ...
       'sauce_jam_v', 'Othrprocessed_v', 'Beveragestotal_v', 'foodtotal_v',
       'foodtotal_q', 'state_1', 'Region', 'fruits_df_tt_v', 'fv_tot',
       'non_veg'],
      dtype='object', length=385)


# Define the dependent variable (non_veg) and independent variables

In [74]:
y = non_veg_data['non_veg']
X = non_veg_data.drop(['non_veg', 'state_1', 'Region'], axis=1)

# Check for non-numeric columns

In [75]:
non_numeric_cols = X.select_dtypes(include=[object]).columns
print("Non-numeric columns:", non_numeric_cols)

Non-numeric columns: Index(['grp'], dtype='object')


# One-Hot Encode non-numeric columns using pd.get_dummies

In [77]:
X_ohe = pd.get_dummies(X[non_numeric_cols], drop_first=True)
X_ohe = pd.get_dummies(X[non_numeric_cols], drop_first=True, prefix=non_numeric_cols)

# Combine numeric and one-hot encoded columns

In [78]:
X_numeric = X.select_dtypes(include=[np.number])
X_combined = pd.concat([X_numeric.reset_index(drop=True), X_ohe.reset_index(drop=True)], axis=1)

# Handle missing values using median imputation


In [83]:
from sklearn.impute import SimpleImputer

In [89]:
imputer = SimpleImputer(strategy="constant", fill_value=0)
X_imputed = imputer.fit_transform(X_combined)

In [94]:
column_names = X_combined.loc[:, X_combined.count() > 0].columns
X_imputed = X_imputed[:, :len(column_names)]  # Select the correct columns
X_combined_imputed = pd.DataFrame(X_imputed, columns=column_names)

# Check for empty columns

In [95]:
empty_cols = X_combined_imputed.columns[X_combined_imputed.count() == 0]
if len(empty_cols) > 0:
    print("Empty columns:", empty_cols)
    X_combined_imputed.drop(columns=empty_cols, inplace=True)

# Verify the shape of the data

In [96]:
print("Shape of X_combined_imputed:", X_combined_imputed.shape)
print("Shape of y:", y.shape)


Shape of X_combined_imputed: (101047, 546)
Shape of y: (101047,)



# Ensure that X_combined_imputed and y have compatible shapes


In [97]:
y = y.reset_index(drop=True)

# Fit the probit model

In [109]:
import statsmodels.api as sm

try:
    probit_model = sm.Probit(y, X_combined_imputed)
    probit_results = probit_model.fit()
    print(probit_results.summary())
except Exception as e:
    print(f"An error occurred: {e}")

C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparat

         Current function value: 1.217031
         Iterations: 35


C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2873: RuntimeWarning: divide by zero encountered in divide
  L = q*self.pdf(q*XB)/self.cdf(q*XB)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2873: RuntimeWarning: invalid value encountered in divide
  L = q*self.pdf(q*XB)/self.cdf(q*XB)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2874: RuntimeWarning: invalid value encountered in multiply
  return np.dot(-L*(L+XB)*X.T,X)
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Bala Vignesh.A\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings

                          Probit Regression Results                           
Dep. Variable:                non_veg   No. Observations:               101047
Model:                         Probit   Df Residuals:                   100591
Method:                           MLE   Df Model:                          455
Date:                Wed, 26 Jun 2024   Pseudo R-squ.:                     inf
Time:                        23:06:26   Log-Likelihood:            -1.2298e+05
converged:                      False   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
slno                                     0.0028        nan        nan        nan         nan         nan
Round_Centre                             0.1783      